In [ ]:
!pip install sentence-transformers pillow requests

In [ ]:
# 라이브러리 import
from sentence_transformers import SentenceTransformer, util
from PIL import Image
import requests
from io import BytesIO
import torch

In [ ]:
# CLIP 모델 로드
model = SentenceTransformer('clip-ViT-B-32')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

0_CLIPModel/pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

0_CLIPModel/model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
# 예시 이미지 URL들 (테스트용)
image_urls = [
    "https://images.unsplash.com/photo-1574158622682-e40e69881006",  # 고양이
    "https://images.unsplash.com/photo-1552053831-71594a27632d",  # 강아지
    "https://images.unsplash.com/photo-1503023345310-bd7c1de61c7d"   # 사람
]

In [ ]:
# 이미지 다운로드 및 로드
images = []
for url in image_urls:
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    images.append(img)

In [ ]:
# 예시 캡션들
captions = [
    "a cute cat",
    "a playful dog",
    "a person standing outdoors"
]

In [ ]:
# 이미지 임베딩 생성
print("\n이미지 임베딩 생성 중...")
image_embeddings = model.encode(images, convert_to_tensor=True)

# 캡션 임베딩 생성
print("\n캡션 임베딩 생성 중...")
caption_embeddings = model.encode(captions, convert_to_tensor=True)

# 유사도 행렬 계산 (코사인 유사도)
print("\n유사도 행렬 계산 중...")
similarity_matrix = util.cos_sim(image_embeddings, caption_embeddings)


이미지 임베딩 생성 중...

캡션 임베딩 생성 중...

유사도 행렬 계산 중...


In [ ]:
# 이미지 이름 정의
image_names = ["고양이 이미지", "강아지 이미지", "사람 이미지"]

# 각 이미지에 가장 잘 맞는 캡션 찾기
print("=== 최적 매칭 결과 ===")
for i, img_name in enumerate(image_names):
    best_match_idx = torch.argmax(similarity_matrix[i]).item()
    best_score = similarity_matrix[i][best_match_idx].item()
    print(f"{img_name} → '{captions[best_match_idx]}' (유사도: {best_score:.4f})")

=== 최적 매칭 결과 ===
고양이 이미지 → 'a cute cat' (유사도: 0.2684)
강아지 이미지 → 'a playful dog' (유사도: 0.2815)
사람 이미지 → 'a person standing outdoors' (유사도: 0.2610)
